In [1]:
import pandas as pd 
import numpy as np

In [38]:
# 流感病毒
diagnoses = pd.read_csv("mimic-iv-2/hosp/diagnoses_icd.csv", header=0)
diagnoses_9e = diagnoses.loc[diagnoses["icd_code"].str.match("487\d"), :]
print(diagnoses_9e.shape)
diagnoses_10e = diagnoses.loc[diagnoses["icd_code"].str.match("J10\d"), :]
print(diagnoses_10e.shape)
diagnoses_vpn = pd.concat((diagnoses_9e, diagnoses_10e), 0)
diagnoses_vpn.to_csv("influ_sofa/diagnoses_flu.csv", header=True, index=True)

(851, 5)
(464, 5)


/tmp/ipykernel_3919697/379714098.py:7: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  diagnoses_vpn = pd.concat((diagnoses_9e, diagnoses_10e), 0)


In [ ]:
# 肺炎链球菌
diagnoses = pd.read_csv("mimic-iv-2/hosp/diagnoses_icd.csv", header=0)
diagnoses_9e = diagnoses.loc[diagnoses["icd_code"].str.match("4823"), :]
print(diagnoses_9e.shape)
diagnoses_10e = diagnoses.loc[diagnoses["icd_code"].str.match("J15[3,4]"), :]
print(diagnoses_10e.shape)
diagnoses_pn = pd.concat((diagnoses_9e, diagnoses_10e), 0)
diagnoses_9e9 = diagnoses.loc[diagnoses["icd_code"].str.match("4829"), :]
diag_culture = pd.merge(diagnoses_9e9,culture)
alist =[i for i in set(diag_culture["org_name"].dropna()) if "STREP" in i]
diag_strep = diag_culture.loc[diag_culture["org_name"].isin(alist),:]
diagnoses_pn = pd.concat((diagnoses_pn, diag_strep[diagnoses_pn.columns]), 0)
diagnoses_pn.to_csv("stre_sofa/diagnoses_strep.csv", header=True, index=True)


# 共同需要的

In [31]:
icu_stay = pd.read_csv("mimic-iv-2/icu/icustays.csv",header=0,)

In [42]:
patients_id = pd.merge(diagnoses_pn, icu_stay).drop_duplicates()
patients_vid = pd.merge(diagnoses_vpn, icu_stay).drop_duplicates()

In [44]:
import os
files = os.listdir("sofa_data")
for file in files:
    df = pd.read_csv("sofa_data/%s" % file, header=0)
    pdfb = pd.merge(patients_id,df)
    pdfb.to_csv("stre_sofa/%s" % file)
    pdfv = pd.merge(patients_vid,df)
    pdfv.to_csv("influ_sofa/%s" % file)

/tmp/ipykernel_3919697/156942697.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("sofa_data/%s" % file, header=0)


In [20]:
import pandas as pd
import numpy as np
sofa = pd.read_csv("stre_sofa/sofa.csv", index_col=0)


In [ ]:
for sd in set(sofa["stay_id"]):
    apsofa = sofa.loc[sofa["stay_id"] == sd]
    for i in range(1, apsofa.shape[0]):
        for j in ['respiration','coagulation', 'liver', 'cardiovascular', 'cns', 'renal']:
            if np.isnan(apsofa[j].iloc[i]) == True:
                if apsofa[j+"_24hours"].iloc[i]==0:
                    apsofa[j+"_24hours"].iloc[i] = apsofa[j+"_24hours"].iloc[i-1]
                    sofa.loc[apsofa.index[i], j+"_24hours"] = apsofa.loc[apsofa.index[i], j+"_24hours"]
                    if sofa.loc[apsofa.index[i], j+"_24hours"] != 0:
                        print(apsofa.index[i], sofa.loc[apsofa.index[i], j+"_24hours"])
